In [1]:
import pandas as pd
import numpy as np
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from sentence_transformers import SentenceTransformer
import pathway as pw

In [52]:
df=pd.read_csv('dataset.csv')



In [3]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=200)

In [4]:
loader_1 = TextLoader("In search of the castaways.txt", encoding="utf-8")
documents_1 = loader_1.load()
loader_2 = TextLoader("The Count of Monte Cristo.txt", encoding="utf-8")
documents_2 = loader_2.load()

In [8]:
chunks_1=splitter.split_documents(documents_1)
chunks_2=splitter.split_documents(documents_2)

In [10]:
chunk_b1=[]
chunk_b2=[]
for i in range(len(chunks_1)):
    chunk_b1.append(chunks_1[i].page_content)
for i in range(len(chunks_2)):
    chunk_b1.append(chunks_2[i].page_content)

In [11]:
mask=df['book_name']=="In Search of the Castaways"
claim_b1=np.array(df[mask]['content'])
mask=df['book_name']==""
claim_b1=np.array(df[mask]['content'])


In [12]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [14]:
chunk_embeddings = model.encode(chunk_b1,convert_to_numpy=True,show_progress_bar=True)
claim_embeddings = model.encode(claim_b1,convert_to_numpy=True,show_progress_bar=True)


Batches:   0%|          | 0/46 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
claim_embeddings[0]

array([ 8.06048587e-02,  5.04271612e-02,  7.01383576e-02,  1.44284889e-02,
       -1.29500730e-02, -7.08252266e-02,  4.99206632e-02, -7.98434392e-02,
        2.24154931e-03, -3.62085402e-02, -2.66883224e-02,  8.06114520e-04,
       -2.55933404e-02,  3.14595923e-02, -1.71355829e-02, -4.86079268e-02,
       -9.39349383e-02,  6.12337664e-02,  2.12097615e-02, -3.44263054e-02,
       -4.95717824e-02, -1.06967194e-02,  1.70336068e-02, -2.20534485e-02,
        8.80729333e-02, -2.03385502e-02,  1.03732189e-02, -7.21783489e-02,
       -1.25996675e-02,  4.11553942e-02,  1.31091513e-02,  8.29578638e-02,
       -6.33436888e-02, -1.01424828e-02, -6.26487704e-03,  3.55114117e-02,
        4.65354100e-02,  3.44410092e-02, -2.84824166e-02, -4.01100479e-02,
        1.29321194e-03, -1.67043712e-02,  2.41234992e-02,  6.01691417e-02,
       -7.70783275e-02, -3.16650756e-02,  9.72927827e-03, -3.51498649e-02,
        3.22775058e-02,  2.64220703e-02, -1.10677533e-01, -4.29192372e-02,
       -4.55867723e-02, -

In [18]:
chunk_embeddings.shape

(1451, 384)

In [32]:
sims = cosine_similarity(claim_embeddings[0].reshape(1,-1),chunk_embeddings)


(1, 1451)

In [33]:
top_k_idx = sims[0].argsort()[-5:][::-1] #selecting top 5 chunks only

In [35]:
top_chunks = [chunk_b1[i] for i in top_k_idx]
top_scores = [sims[0][i] for i in top_k_idx]

# Heuristics code